<a href="https://colab.research.google.com/github/AoShuang92/PhD_tutorial/blob/main/LlaVA_Img_Text_Embedding_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers==4.36.0
!pip install -q bitsandbytes==0.41.3 accelerate==0.25.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 97.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 23.0 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
from transformers import AutoProcessor, LlavaForConditionalGeneration, BitsAndBytesConfig
from PIL import Image
import requests
import torch.nn.functional as F
from typing import List, Optional, Tuple, Union

class LlaVA_Img_Text_Embedding(nn.Module):
    def __init__(self, ):
        super(LlaVA_Img_Text_Embedding, self).__init__()
        kwargs = {"device_map": "auto"}
        kwargs['load_in_4bit'] = True
        kwargs['quantization_config'] = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type='nf4'
        )
        self.model_llava = LlavaForConditionalGeneration.from_pretrained("llava-hf/llava-1.5-7b-hf",**kwargs)
        self.get_input_embeddings = self.model_llava.get_input_embeddings
        self.vision_tower = self.model_llava.vision_tower
        self.multi_modal_projector = self.model_llava.multi_modal_projector
        self._merge_input_ids_with_image_features = self.model_llava._merge_input_ids_with_image_features

    def forward(
            self,
            input_ids: torch.LongTensor = None,
            pixel_values: torch.FloatTensor = None,
            attention_mask: Optional[torch.Tensor] = None,
            position_ids: Optional[torch.LongTensor] = None,

        ):
        vision_feature_select_strategy = "default"
        # 1. Extra the input embeddings
        inputs_embeds = self.get_input_embeddings()(input_ids)
        # 2. Merge text and images
        if pixel_values is not None and input_ids.shape[1] != 1:
            selected_image_feature = self.vision_tower(pixel_values).last_hidden_state

            if vision_feature_select_strategy == "default":
                selected_image_feature = selected_image_feature[:, 1:]
            elif vision_feature_select_strategy == "full":
                selected_image_feature = selected_image_feature
            else:
                raise ValueError(
                    f"Unexpected select feature strategy: {self.config.vision_feature_select_strategy}"
                )

            image_features = self.multi_modal_projector(selected_image_feature)

            print('image_features:', image_features.shape, 'text input_ids:', input_ids.shape, input_ids)
            inputs_embeds, attention_mask, position_ids = self._merge_input_ids_with_image_features(
                image_features, inputs_embeds, input_ids, attention_mask, position_ids
            )


        return inputs_embeds, attention_mask, position_ids

prompt = "<image>\nUSER: What's the content of the image?\nASSISTANT:"
url = "https://www.ilankelman.org/stopsigns/australia.jpg"
image = Image.open(requests.get(url, stream=True).raw)
processor = AutoProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf")
inputs = processor(text=prompt, images=image, return_tensors="pt")
# inputs['input_ids'] = inputs['input_ids'].cuda()
for key, val in inputs.items():
    inputs[key] = val.cuda()

img_text_embeder = LlaVA_Img_Text_Embedding().cuda()
inputs_embeds, attention_mask, position_ids = img_text_embeder(**inputs)

print(inputs_embeds.shape)


/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `fo

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


image_features: torch.Size([1, 576, 4096]) text input_ids: torch.Size([1, 20]) tensor([[    1, 32000, 29871,    13, 11889, 29901,  1724, 29915, 29879,   278,
          2793,   310,   278,  1967, 29973,    13, 22933,  9047, 13566, 29901]],
       device='cuda:0')
torch.Size([1, 595, 4096])


In [ ]:
inputs_embeds

tensor([[[ 0.0045, -0.0038,  0.0017,  ..., -0.0088,  0.0025, -0.0025],
         [ 0.0718,  0.7710, -0.4734,  ..., -0.2423,  0.6743, -0.1895],
         [ 0.5903,  0.4768, -0.6670,  ..., -0.4971,  0.5259, -0.3545],
         ...,
         [-0.0187, -0.0017,  0.0177,  ...,  0.0238,  0.0052,  0.0101],
         [ 0.0066, -0.0161,  0.0117,  ..., -0.0103,  0.0148,  0.0073],
         [ 0.0039,  0.0015,  0.0055,  ..., -0.0042,  0.0151,  0.0024]]],
       device='cuda:0', dtype=torch.float16, grad_fn=<IndexPutBackward0>)

In [ ]:
kwargs = {"device_map": "auto"}
kwargs['load_in_4bit'] = True
kwargs['quantization_config'] = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4'
)
model_llava = LlavaForConditionalGeneration.from_pretrained("llava-hf/llava-1.5-7b-hf",**kwargs)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
prompt = "<image>\nUSER: What's the content of the image?\nASSISTANT:"
url = "https://www.ilankelman.org/stopsigns/australia.jpg"
image = Image.open(requests.get(url, stream=True).raw)
processor = AutoProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf")
inputs = processor(text=prompt, images=image, return_tensors="pt")
# inputs['input_ids'] = inputs['input_ids'].cuda()
for key, val in inputs.items():
    inputs[key] = val.cuda()
    print(key)

print(inputs['pixel_values'].shape, inputs['input_ids'].shape, inputs['input_ids'])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


input_ids
attention_mask
pixel_values
torch.Size([1, 3, 336, 336]) torch.Size([1, 20]) tensor([[    1, 32000, 29871,    13, 11889, 29901,  1724, 29915, 29879,   278,
          2793,   310,   278,  1967, 29973,    13, 22933,  9047, 13566, 29901]],
       device='cuda:0')


In [ ]:


get_input_embeddings = model_llava.get_input_embeddings
vision_tower = model_llava.vision_tower
multi_modal_projector = model_llava.multi_modal_projector
_merge_input_ids_with_image_features = model_llava._merge_input_ids_with_image_features
inputs_embeds = get_input_embeddings()(inputs['input_ids'])
print(inputs_embeds.shape, inputs_embeds)

torch.Size([1, 20, 4096]) tensor([[[ 0.0045, -0.0038,  0.0017,  ..., -0.0088,  0.0025, -0.0025],
         [ 0.0007,  0.0006, -0.0005,  ..., -0.0006, -0.0001, -0.0005],
         [-0.0012,  0.0013, -0.0127,  ...,  0.0026, -0.0012, -0.0053],
         ...,
         [-0.0187, -0.0017,  0.0177,  ...,  0.0238,  0.0052,  0.0101],
         [ 0.0066, -0.0161,  0.0117,  ..., -0.0103,  0.0148,  0.0073],
         [ 0.0039,  0.0015,  0.0055,  ..., -0.0042,  0.0151,  0.0024]]],
       device='cuda:0', dtype=torch.float16, grad_fn=<EmbeddingBackward0>)


In [ ]:
selected_image_feature = vision_tower(inputs['pixel_values']).last_hidden_state
print(selected_image_feature.shape, selected_image_feature)

selected_image_feature = selected_image_feature[:, 1:]
print(selected_image_feature.shape, selected_image_feature)

torch.Size([1, 577, 1024]) tensor([[[-0.4695, -0.2588, -0.0212,  ..., -0.1227, -0.8413,  0.4075],
         [ 0.5752,  1.1084,  0.0914,  ...,  1.5645,  0.1642,  0.9097],
         [ 0.2900,  0.3425,  0.7036,  ...,  1.6426, -0.4028,  0.5298],
         ...,
         [ 1.8662,  1.0088, -0.2004,  ...,  1.8193, -0.8154,  0.8516],
         [ 0.3364,  0.2842,  0.9233,  ...,  0.4353,  0.3420, -0.4443],
         [ 0.0479,  0.2493,  0.5264,  ...,  0.0032,  0.2112,  0.1550]]],
       device='cuda:0', dtype=torch.float16, grad_fn=<AddBackward0>)
torch.Size([1, 576, 1024]) tensor([[[ 0.5752,  1.1084,  0.0914,  ...,  1.5645,  0.1642,  0.9097],
         [ 0.2900,  0.3425,  0.7036,  ...,  1.6426, -0.4028,  0.5298],
         [ 0.4731, -0.1261,  0.5146,  ..., -0.2083,  0.1995, -0.4390],
         ...,
         [ 1.8662,  1.0088, -0.2004,  ...,  1.8193, -0.8154,  0.8516],
         [ 0.3364,  0.2842,  0.9233,  ...,  0.4353,  0.3420, -0.4443],
         [ 0.0479,  0.2493,  0.5264,  ...,  0.0032,  0.2112,  0.15

In [ ]:
image_features = multi_modal_projector(selected_image_feature)
print(image_features.shape, image_features)

torch.Size([1, 576, 4096]) tensor([[[ 0.0718,  0.7710, -0.4734,  ..., -0.2423,  0.6743, -0.1895],
         [ 0.5903,  0.4768, -0.6670,  ..., -0.4971,  0.5259, -0.3545],
         [ 0.5405,  0.4556, -0.1338,  ..., -0.3860,  1.0166, -0.4998],
         ...,
         [ 0.8389,  0.0048, -1.1475,  ..., -0.2019,  0.4473,  0.0886],
         [ 0.6997,  0.1176, -0.7114,  ..., -0.6050,  0.8491,  0.0121],
         [ 0.0582,  0.4683, -0.4680,  ..., -0.5923,  0.4304, -0.4587]]],
       device='cuda:0', dtype=torch.float16, grad_fn=<MatMul4BitBackward>)


In [ ]:
print(image_features.shape, inputs_embeds.shape)
inputs_embeds, attention_mask, position_ids = _merge_input_ids_with_image_features(
                image_features, inputs_embeds, inputs['input_ids'], attention_mask, position_ids
            )

print(inputs_embeds.shape, attention_mask.shape, position_ids.shape, inputs['input_ids'].shape)

torch.Size([1, 576, 4096]) torch.Size([1, 20, 4096])
torch.Size([1, 595, 4096]) torch.Size([1, 595]) torch.Size([1, 595]) torch.Size([1, 20])


In [ ]:
model_llava.config.vision_config

CLIPVisionConfig {
  "attention_dropout": 0.0,
  "hidden_act": "quick_gelu",
  "hidden_size": 1024,
  "image_size": 336,
  "initializer_factor": 1.0,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "model_type": "clip_vision_model",
  "num_attention_heads": 16,
  "num_channels": 3,
  "num_hidden_layers": 24,
  "patch_size": 14,
  "projection_dim": 768,
  "transformers_version": "4.36.0",
  "vocab_size": 32000
}

In [ ]:
model_llava.config.projector_hidden_act

'gelu'

In [ ]:
def merge_input_ids_with_image_features( image_features, inputs_embeds, input_ids, attention_mask, position_ids):
    # print('attention_mask, position_ids', attention_mask.size(), position_ids.size()) [1, 595]

    num_images, num_image_patches, embed_dim = image_features.shape #[1, 576, 4096]
    batch_size, sequence_length = input_ids.shape #[1, 20]
    # print(batch_size, sequence_length) 1 20

    left_padding = not torch.sum(input_ids[:, -1] == torch.tensor(model_llava.pad_token_id))
    # print('left_padding',left_padding, input_ids[:, -1], torch.tensor(model_llava.pad_token_id))
    #True, tensor([29901], device='cuda:0') tensor(32001)


    # 1. Create a mask to know where special image tokens are
    special_image_token_mask = input_ids == model_llava.config.image_token_index
    # print('special_image_token_mask', special_image_token_mask)
    # print('model_llava.config.image_token_index', model_llava.config.image_token_index) 32000
    # print('special_image_token_mask', special_image_token_mask) #[1, 20] with all False, but position 1 is True

    num_special_image_tokens = torch.sum(special_image_token_mask, dim=-1)
    # print('num_special_image_tokens', num_special_image_tokens) [1]

    # Compute the maximum embed dimension
    max_embed_dim = (num_special_image_tokens.max() * (num_image_patches - 1)) + sequence_length
    # print('max_embed_dim', max_embed_dim) 595

    batch_indices, non_image_indices = torch.where(input_ids != model_llava.config.image_token_index)
    # print('batch_indices', batch_indices.shape, 'non_image_indices', non_image_indices.shape) shape [19]
    # batch_indices: shape [19], with all 0
    # non_image_indices tensor([ 0,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

    # 2. Compute the positions where text should be written (include the first position)
    # Calculate new positions for text tokens in merged image-text sequence.
    # `special_image_token_mask` identifies image tokens. Each image token will be replaced by `nb_text_tokens_per_images - 1` text tokens.
    # `torch.cumsum` computes how each image token shifts subsequent text token positions.
    # - 1 to adjust for zero-based indexing, as `cumsum` inherently increases indices by one.

    new_token_positions = torch.cumsum((special_image_token_mask * (num_image_patches - 1) + 1), -1) - 1
    # print('temp', (special_image_token_mask * (num_image_patches - 1) + 1).shape ) [1, 20]
    # [  1, 576,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1, 1,   1,   1,   1,   1,   1]
    # print('temp1', torch.cumsum((special_image_token_mask * (num_image_patches - 1) + 1), -1))
    # [  1, 577, 578, 579, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595]
    # print('new_token_positions', new_token_positions.shape) [1, 20]
    # [  0, 576, 577, 578, 579, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594]



    nb_image_pad = max_embed_dim - 1 - new_token_positions[:, -1]
    # print('nb_image_pad', nb_image_pad, max_embed_dim,new_token_positions[:, -1] ) [0], 595, [594]

    if left_padding:
        new_token_positions += nb_image_pad[:, None]  # offset for left padding
    # print('new_token_positions', new_token_positions)
    # [  0, 576, 577, 578, 579, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594] Not changed

    text_to_overwrite = new_token_positions[batch_indices, non_image_indices]
    # print('text_to_overwrite', text_to_overwrite.shape) [19]
    # [  0, 577, 578, 579, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594]

    # 3. Create the full embedding, already padded to the maximum position
    final_embedding = torch.zeros(
        batch_size, max_embed_dim, embed_dim, dtype=inputs_embeds.dtype, device=inputs_embeds.device
    )
    # print('final_embedding', final_embedding.size()) [1, 595, 4096]
    final_attention_mask = torch.zeros(
        batch_size, max_embed_dim, dtype=attention_mask.dtype, device=inputs_embeds.device
    )
    # print('final_attention_mask', final_attention_mask.size()) [1, 595]

    # 4. Fill the embeddings based on the mask. If we have ["hey" "<image>", "how", "are"]
    # we need to index copy on [0, 577, 578, 579] for the text and [1:576] for the image features
    final_embedding[batch_indices, text_to_overwrite] = inputs_embeds[batch_indices, non_image_indices]
    final_attention_mask[batch_indices, text_to_overwrite] = attention_mask[batch_indices, non_image_indices]
    # print(final_embedding[batch_indices, text_to_overwrite].shape)[19, 4096]
    # print(final_attention_mask[batch_indices, text_to_overwrite].shape) [19]

    # 5. Fill the embeddings corresponding to the images. Anything that is still zeros needs filling
    image_to_overwrite = torch.all(final_embedding == 0, dim=-1)

    # print('image_to_overwrite', image_to_overwrite.size()) [1, 595]
    # print(image_to_overwrite) 0 position is False, from 577 to the end position is False

    # print(image_to_overwrite.cumsum(-1) - 1)
    # &=: bitwise AND operator, assigns the result back to the variable on the left-hand side
    # print(nb_image_pad[:, None]) [0]
    image_to_overwrite &= image_to_overwrite.cumsum(-1) - 1 >= nb_image_pad[:, None]
    # print('image_to_overwrite', image_to_overwrite) the first position and all text position are False

    if image_to_overwrite.sum() != image_features.shape[:-1].numel():
        raise ValueError(
            f"The input provided to the model are wrong. The number of image tokens is {torch.sum(special_image_token_mask)} while"
            f" the number of image given to the model is {num_images}. This prevents correct indexing and breaks batch generation."
        )

    final_embedding[image_to_overwrite] = image_features.contiguous().reshape(-1, embed_dim)
    # print(final_embedding[image_to_overwrite].size()) [576, 4096]

    #|=: bitwise OR operator;assigns the result back to the variable on the left-hand side
    final_attention_mask |= image_to_overwrite
    # print('final_attention_mask', final_attention_mask.size()) [1, 595]

    position_ids = (final_attention_mask.cumsum(-1) - 1).masked_fill_((final_attention_mask == 0), 1)
    # print('position_ids', position_ids.size()) [1, 595]

    return final_embedding, final_attention_mask, position_ids

print("original input",image_features.shape, inputs_embeds.shape)
inputs_embeds, attention_mask, position_ids = merge_input_ids_with_image_features(
                image_features, inputs_embeds, inputs['input_ids'], attention_mask, position_ids
            )

print(inputs_embeds.shape, inputs_embeds)


original input torch.Size([1, 576, 4096]) torch.Size([1, 595, 4096])
torch.Size([1, 595, 4096]) tensor([[[ 0.0045, -0.0038,  0.0017,  ..., -0.0088,  0.0025, -0.0025],
         [ 0.0718,  0.7710, -0.4734,  ..., -0.2423,  0.6743, -0.1895],
         [ 0.5903,  0.4768, -0.6670,  ..., -0.4971,  0.5259, -0.3545],
         ...,
         [ 1.4023,  0.5166, -0.0838,  ..., -0.8745,  0.1522, -0.6401],
         [ 0.4265,  0.7480, -1.0049,  ..., -0.1869,  0.7275,  0.0514],
         [-0.0573,  0.6553, -0.5039,  ..., -0.5781,  0.2234,  0.0302]]],
       device='cuda:0', dtype=torch.float16, grad_fn=<IndexPutBackward0>)


In [ ]:
model_llava.pad_token_id, model_llava.config.image_token_index

(32001, 32000)

In [ ]:
temp = torch.tensor(([[1,2,3], [4,5,6]] , [[7,8,0], [0,0,0]]))
temp.size()
temp[-1,-1,:]
# temp_ = temp.unsqueeze(0)
# temp_.size()
# temp_[:,-1,:]

tensor([0, 0, 0])

In [ ]:
!python -V

Python 3.10.12


In [ ]:
model_llava.config.vision_config

/usr/local/bin/python


In [ ]:
from transformers import AutoTokenizer, CLIPVisionModel
model = CLIPVisionModel.from_pretrained("openai/clip-vit-base-patch32").cuda()
tokenizer = AutoTokenizer.from_pretrained("openai/clip-vit-base-patch32")
image = inputs['pixel_values']
image = transforms.Resize((224,224), interpolation=Image.BICUBIC)(image)
print("image", image.size())

inputs_img = {'pixel_values':0}
inputs_img['pixel_values'] = image
# model.get_image_features
image_features = model(**inputs_img)
image_features.last_hidden_state.size(),

image torch.Size([1, 3, 224, 224])


(torch.Size([1, 50, 768]),)

In [ ]:
model_llava.config.text_config

LlamaConfig {
  "_name_or_path": "lmsys/vicuna-7b-v1.5",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 4096,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.36.0",
  "use_cache": true,
  "vocab_size": 32064
}

In [ ]:
CLIPVisionConfig {
  "attention_dropout": 0.0,
  "hidden_act": "quick_gelu",
  "hidden_size": 1024,
  "image_size": 336,
  "initializer_factor": 1.0,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "model_type": "clip_vision_model",
  "num_attention_heads": 16,
  "num_channels": 3,
  "num_hidden_layers": 24,
  "patch_size": 14,
  "projection_dim": 768,
  "transformers_version": "4.36.0",
  "vocab_size": 32000
}

Fuse image and text embedding

In [3]:
import torch
import torch.nn as nn
from transformers import AutoProcessor, LlavaForConditionalGeneration, BitsAndBytesConfig
from PIL import Image
import requests
import torch.nn.functional as F
from typing import List, Optional, Tuple, Union

prompt = "<image>\nUSER: What's the content of the image?\nASSISTANT:"
url = "https://www.ilankelman.org/stopsigns/australia.jpg"
image = Image.open(requests.get(url, stream=True).raw)
processor = AutoProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf")
inputs = processor(text=prompt, images=image, return_tensors="pt")
# inputs['input_ids'] = inputs['input_ids'].cuda()
for key, val in inputs.items():
    inputs[key] = val.cuda()
    print(key)

print(inputs['pixel_values'].shape, inputs['input_ids'].shape, inputs['input_ids'])

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/505 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/41.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


input_ids
attention_mask
pixel_values
torch.Size([1, 3, 336, 336]) torch.Size([1, 20]) tensor([[    1, 32000, 29871,    13, 11889, 29901,  1724, 29915, 29879,   278,
          2793,   310,   278,  1967, 29973,    13, 22933,  9047, 13566, 29901]],
       device='cuda:0')


In [21]:
from transformers import AutoTokenizer, CLIPVisionModel
from torchvision import transforms
from torch import nn
from collections import OrderedDict
from transformers.activations import ACT2FN

config = {}
config["hidden_size"] = 1024
config["hidden_size_text"] = 4096

config["projector_hidden_act"] = 'gelu'


class LlavaMultiModalProjector(nn.Module):
    def __init__(self, config):
        super().__init__()

        self.linear_1 = nn.Linear(config["hidden_size"], config["hidden_size_text"], bias=True)
        self.act = ACT2FN[config["projector_hidden_act"]]
        self.linear_2 = nn.Linear(config["hidden_size_text"], config["hidden_size_text"], bias=True)

    def forward(self, image_features):
        hidden_states = self.linear_1(image_features)
        hidden_states = self.act(hidden_states)
        hidden_states = self.linear_2(hidden_states)
        return hidden_states


model = CLIPVisionModel.from_pretrained("openai/clip-vit-large-patch14-336").cuda()
# tokenizer = AutoTokenizer.from_pretrained("openai/clip-vit-base-patch14")
image = inputs['pixel_values']
image = transforms.Resize((336,336), interpolation=Image.BICUBIC)(image)
print("image", image.size())

inputs_img = {'pixel_values':0}
inputs_img['pixel_values'] = image
# model.get_image_features

projector = LlavaMultiModalProjector(config).cuda()

image_features = model(**inputs_img)
selected_image_features = image_features.last_hidden_state[:, 1:]
print('image_features',image_features.last_hidden_state.size(), selected_image_features.size())

img_emd = projector(selected_image_features)
img_emd.size()

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--openai--clip-vit-large-patch14-336/snapshots/ce19dc912ca5cd21c8a653c79e251e808ccabcd1/config.json
Model config CLIPVisionConfig {
  "attention_dropout": 0.0,
  "dropout": 0.0,
  "hidden_act": "quick_gelu",
  "hidden_size": 1024,
  "image_size": 336,
  "initializer_factor": 1.0,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "model_type": "clip_vision_model",
  "num_attention_heads": 16,
  "num_channels": 3,
  "num_hidden_layers": 24,
  "patch_size": 14,
  "projection_dim": 768,
  "transformers_version": "4.36.0"
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--openai--clip-vit-large-patch14-336/snapshots/ce19dc912ca5cd21c8a653c79e251e808ccabcd1/pytorch_model.bin
Some weights of the model checkpoint at openai/clip-vit-large-patch14-336 were not used when initializing CLIPVisionModel: ['text_model.final_layer_norm.bi

image torch.Size([1, 3, 336, 336])
image_features torch.Size([1, 577, 1024]) torch.Size([1, 576, 1024])


torch.Size([1, 576, 4096])